In [4]:
import cv2
import numpy as np
import pandas as pd
import easyocr
from ultralytics import YOLO
from collections import deque
from datetime import datetime

# Load YOLOv8 model
model = YOLO('/Users/prakhyath/Desktop/capstone project/best.pt')

# Initialize the CSV file
csv_file = 'output6.csv'
columns = ['Date', 'Time', 'Vehicle Class', 'Direction']
df = pd.DataFrame(columns=columns)

# Tracker dictionary to track vehicle positions (ID: centroid)
trackers = {}
vehicle_id = 0
track_memory = 50  # How long to keep track of a vehicle's centroid (for direction detection)

# ROI coordinates for date and time cropping
roi_coords = [
    [1260, 3],
    [1901, 12],
    [1894, 72],
    [1257, 62]
]

# Extract bounding box coordinates from ROI
x1, y1 = roi_coords[0][0], roi_coords[0][1]
x2, y2 = roi_coords[2][0], roi_coords[2][1]

# Instantiate EasyOCR Reader
reader = easyocr.Reader(['en'])

# Function to process vehicles and avoid duplicate storage
def process_vehicles(frame, results, date, time):
    global df, vehicle_id

    new_rows = []

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            vehicle_class = model.names[class_id]

            x1, y1, x2, y2 = box.xyxy[0].tolist()
            centroid_x = int((x1 + x2) / 2)
            centroid_y = int((y1 + y2) / 2)

            vehicle_detected = False
            for v_id, track_data in trackers.items():
                prev_centroid = track_data['centroid']
                if np.linalg.norm(np.array([centroid_x, centroid_y]) - np.array(prev_centroid)) < 50:
                    trackers[v_id]['centroid'] = (centroid_x, centroid_y)
                    vehicle_detected = True

                    track_y = deque(trackers[v_id]['y_coords'], maxlen=track_memory)
                    track_y.append(centroid_y)
                    trackers[v_id]['y_coords'] = track_y
                    direction = get_direction(track_y)

                    # Ensure the vehicle hasn't already been stored in the CSV for this session
                    if not track_data['logged']:
                        new_row = {
                            'Date': str(date),  # Already formatted as YYYY-MM-DD
                            'Time': str(time),  # Already formatted as HH:MM:SS
                            'Vehicle Class': vehicle_class,
                            'Direction': direction
                        }
                        new_rows.append(new_row)
                        trackers[v_id]['logged'] = True  # Mark as logged

                    break

            # If vehicle was not detected previously, assign a new ID
            if not vehicle_detected:
                trackers[vehicle_id] = {
                    'centroid': (centroid_x, centroid_y),
                    'y_coords': deque([centroid_y], maxlen=track_memory),
                    'logged': False  # Not yet logged in CSV
                }
                vehicle_id += 1

    if new_rows:
        new_df = pd.DataFrame(new_rows)
        df = pd.concat([df, new_df], ignore_index=True)

        # Save DataFrame to CSV
        df.to_csv(csv_file, index=False)

# Function to detect direction (up or down)
def get_direction(y_coords):
    if len(y_coords) < 2:
        return 'Unknown'
    if y_coords[0] - y_coords[-1] > 0:
        return 'up'
    elif y_coords[-1] - y_coords[0] > 0:
        return 'down'
    return 'Unknown'

# Function to extract date and time from a cropped video frame
def extract_date_time(frame):
    results = reader.readtext(frame, detail=0)

    if results and len(results) >= 2:
        # Assuming the OCR returns date in 'YYYY:MM:DD' format and time in 'HH:MM:SS' or similar
        date_str = results[0].replace(':', '-')
        time_str = results[1]

        # Ensure proper formatting of the date (YYYY-MM-DD) and time (HH:MM:SS)
        try:
            date = datetime.strptime(date_str, "%Y-%m-%d").date()  # Format date
            time = datetime.strptime(time_str, "%H:%M:%S").time()  # Format time
        except ValueError:
            # If the parsing fails, return None
            return None, None

        return date, time

    return None, None

# Main function to process the video
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Crop the frame using the provided ROI coordinates for date and time
        cropped_frame = frame[y1:y2, x1:x2]

        # Extract date and time from the cropped frame
        date, time = extract_date_time(cropped_frame)

        if date and time:
            # Perform vehicle detection and tracking
            results = model(frame)

            # Process vehicles and save them to CSV
            process_vehicles(frame, results, date, time)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the main function with your video path
process_video('/Users/prakhyath/Desktop/capstone project/obj detection/18th_Crs_BsStp_JN_FIX_2_time_2024-05-14T07-30-02_000.mp4')



0: 288x512 1 Two-Wheeler, 94.7ms
Speed: 13.3ms preprocess, 94.7ms inference, 14.8ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 74.0ms
Speed: 1.0ms preprocess, 74.0ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 68.0ms
Speed: 1.4ms preprocess, 68.0ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 62.9ms
Speed: 1.1ms preprocess, 62.9ms inference, 0.3ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 65.5ms
Speed: 1.2ms preprocess, 65.5ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 60.2ms
Speed: 2.3ms preprocess, 60.2ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 81.4ms
Speed: 1.6ms preprocess, 81.4ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 Two-Wheeler, 138.5ms
Speed: 1.1ms preprocess, 138.5ms inference, 0.

KeyboardInterrupt: 